# Bayesian AGN Decomposition Analysis for SDSS Spectra - Multi Spectra (Version 7.3.0) 

###  Remington O. Sexton (UCR), William Matzko (GMU), Nicholas Darden (UCR), 

some would say its pretty BADASS...

In [ ]:
import glob
import time
import natsort
from IPython.display import clear_output
import multiprocessing as mp
import os
import psutil
# To see full list of imported packages and modules, see 
# badass_7_0_0.py file.
import badass_v7_3_0 as badass # <<<--- Import BADASS here

### BADASS Options

In [ ]:
################################## Fit Options #################################
# Fitting Parameters
fit_reg       = (4400,5800) # Fitting region; Indo-US Library=(3460,9464)
good_thresh   = 0.0 # percentage of "good" pixels required in fig_reg for fit.
# Outflow Testing Parameters
test_outflows      = True 
outflow_test_niter = 10 # number of monte carlo iterations for outflows
# Maximum Likelihood Fitting for Final Model Parameters
max_like_niter = 10 # number of maximum likelihood iterations
# LOSVD parameters
min_sn_losvd  = 20  # minimum S/N threshold for fitting the LOSVD
################################################################################

########################### MCMC algorithm parameters ##########################
mcmc_fit      = True # Perform robust fitting using emcee
nwalkers      = 100  # Number of emcee walkers; min = 2 x N_parameters
auto_stop     = True # Automatic stop using autocorrelation analysis
conv_type     = 'median' # 'median', 'mean', 'all', or (tuple) of parameters
min_samp      = 2500  # min number of iterations for sampling post-convergence
ncor_times    = 10.0  # number of autocorrelation times for convergence
autocorr_tol  = 10.0  # percent tolerance between checking autocorr. times
write_iter    = 100   # write/check autocorrelation times interval
write_thresh  = 100   # when to start writing/checking parameters
burn_in       = 17500 # burn-in if max_iter is reached
min_iter      = 100   # min number of iterations before stopping
max_iter      = 20000 # max number of MCMC iterations
################################################################################

############################# Fit component options ############################
fit_feii      = True # fit broad and narrow FeII emission
fit_losvd     = True # fit LOSVD (stellar kinematics) in final model
fit_host      = True # fit host-galaxy using template (if fit_LOSVD turned off)
fit_power     = True # fit AGN power-law continuum
fit_broad     = True # fit broad lines (Type 1 AGN)
fit_narrow    = True # fit narrow lines
fit_outflows  = True # fit outflows;
tie_narrow    = False  # tie narrow widths (don't do this)
################################################################################

############################# Outflow Test options #############################
# Here one can choose how outflows are fit and tested for 
# Amp. test   : outflow amp. must be N-sigma greater than noise
# FWHM test   : outflow must have greater FWHM than core comp by N-sigma
# VOFF test   : outflow must have a larger offset than core relative to rest;
#               picks out only blueshifted outflows by N-sigma
# Resid. test : there must be a measurable difference in residuals by N-sigma
# Bounds. test: if paramters of fit reach bounds by N-sigma, 
#               consider it a bad fit.
outflow_test_pars={
'amp_test':{'test':True,'nsigma':1.0}, # Amplitude-over-noise by n-sigma
'fwhm_test':{'test':True,'nsigma':1.0}, # FWHM difference by n-sigma
'voff_test':{'test':False,'nsigma':1.0}, # blueshift voff from core by n-sigma
'resid_test':{'test':True,'nsigma':1.0}, # residual difference by n-sigma
'bounds_test':{'test':True,'nsigma':1.0} # within bounds by n-sigma
}
################################################################################

############################### Plotting options ###############################
plot_param_hist = True  # Plot MCMC histograms and chains for each parameter
plot_flux_hist  = True  # Plot MCMC hist. and chains for component fluxes
plot_lum_hist   = True  # Plot MCMC hist. and chains for component luminosities
plot_mbh_hist   = True  # Plot MCMC hist. for estimated AGN lum. and BH masses
plot_corner     = False # Plot corner plot of relevant parameters; Corner plots 
                        # of free paramters can be quite large require a PDF 
                        # output, and have significant time and space overhead, 
                        # so we set this to False by default. 
plot_bpt        = True  # Plot BPT diagram 
write_chain     = False # Write MCMC chains for all paramters, fluxes, and
                        # luminosities to a FITS table We set this to false 
                        # because MCMC_chains.FITS file can become very large, 
                        # especially  if you are running multiple objects.  
                        # You only need this if you want to reconstruct chains 
                        # and histograms. 
print_output    = False  # prints steps of fitting process in Jupyter output
################################################################################

############################ Multiprocessing options ###########################
# If fitting single object at a time (no for loops!) then one can set threads>1
# If one wants to fit objects sequentially (one after another), it must be set 
# to threads=1, and must use multiprocessing to spawn subprocesses without 
# significant memory leaks. 
threads = 1 # number of processes per object
################################################################################


### Run BADASS using `multiprocessing.pool` to fit $N$ spectra simultaenously

The following is shows how to fit multiple SDSS spectra simultaneously using `multiprocessing.pool()`.  The number of spectra $N$ you can fit simultaneously ultimately depends on the number of CPU cores and RAM available on your system.

#### Directory Structure

In [ ]:
########################## Directory Structure #################################
spec_dir = 'examples/' # folder with spectra in it
ppxf_dir = 'badass_data_files/' # support files
temp_dir = ppxf_dir+'indo_us_library' # stellar templates
# Get full list of spectrum folders; these will be the working directories
spec_loc = natsort.natsorted( glob.glob(spec_dir+'*') )
################################################################################

### Multiprocessing Options

In [ ]:
# Get number of cpus; 
# nprocesses should not exceed ncpu
ncpu = mp.cpu_count() 
print(' Number of CPUs available = %d' % (ncpu) )
# nprocesses is the number of spectra we want to fit simultaneously
nprocesses = 4
# Number of objects we want to fit in total.
nobj = range(4)#range(len(spec_loc))
# For this to work, each multiprocessing process cannot have child
# processes (i.e., emcee multithreading), so we set threads=1
threads = 1
# Fitting multiple objects simultaneouly means some will finish faster 
# than others, which means printing output makes a mess, so we 
# set print_output=False
print_output = False

In [ ]:
# Print memory of python process at the start
process = psutil.Process(os.getpid())
print('{0:<40}{1:<30.8f}'.format('Start process memory:', process.memory_info().rss/1.0e+9))

def badass_mp(nobj):

    work_dir = spec_loc[nobj]+'/' # working directory
    # Set up run ('MCMC_output_#') directory
    run_dir,prev_dir = badass.setup_dirs(work_dir,print_output=print_output) 
    file = glob.glob(work_dir+'*.fits')[0] # Get name of FITS spectra file
    #
    # Call the main function in BADASS
    badass.run_BADASS(file,run_dir,temp_dir,
                      fit_reg=fit_reg, 
                      good_thresh=good_thresh,
                      test_outflows=test_outflows, 
                      outflow_test_niter=outflow_test_niter,
                      max_like_niter=max_like_niter, 
                      min_sn_losvd=min_sn_losvd,
                      mcmc_fit=mcmc_fit, 
                      nwalkers=nwalkers, 
                      auto_stop=auto_stop, 
                      conv_type=conv_type, 
                      min_samp=min_samp, 
                      ncor_times=ncor_times, 
                      autocorr_tol=autocorr_tol,
                      write_iter=write_iter, 
                      write_thresh=write_thresh, 
                      burn_in=burn_in, 
                      min_iter=min_iter, 
                      max_iter=max_iter,
                      fit_feii=fit_feii, 
                      fit_losvd=fit_losvd, 
                      fit_host=fit_host, 
                      fit_power=fit_power, 
                      fit_broad=fit_broad, 
                      fit_narrow=fit_narrow, 
                      fit_outflows=fit_outflows, 
                      tie_narrow=tie_narrow,
                      outflow_test_pars=outflow_test_pars,
                      plot_param_hist=plot_param_hist, 
                      plot_flux_hist=plot_flux_hist, 
                      plot_lum_hist=plot_lum_hist,
                      plot_mbh_hist=plot_mbh_hist, 
                      plot_corner=plot_corner,
                      plot_bpt=plot_bpt,
                      write_chain=write_chain,
                      print_output=print_output,
                      threads=threads)
    #
if __name__ == '__main__':
    # Initialize pool
    pool = mp.Pool(processes=nprocesses,maxtasksperchild=1)
    results = pool.map(badass_mp, nobj, chunksize=1)
    pool.close()
    pool.join()

# Print memory of python process at the end
process = psutil.Process(os.getpid())
print('{0:<40}{1:<30.8f}'.format(' End process memory:', process.memory_info().rss/1.0e+9))